In [41]:

import pandas as pd
import plotly.express as px
import altair as alt
# from sqlalchemy import create_engine
# import secrets
import numpy as np
from datetime import datetime

In [42]:
df = pd.read_csv(r"data/all_sales_data.csv")
# ### MASTER DATA ###
all_sales = df.copy()

all_sales = all_sales[all_sales.market_segment != 'Samples']

# invoice date cleanup
all_sales['date'] = pd.to_datetime(all_sales['date'])
all_sales['date'] = all_sales['date'].dt.normalize()
all_sales['date'] = all_sales['date'].dt.floor('D')

In [43]:
# current_date = datetime.today.strftime('%Y-%m-%d')
# # import datetime
# year_ago_today = datetime.today - datetime.timedelta(days=365)
# # year_ago_today.strftime('%Y-%m-%d')

# # ---- TOP KPI's Row ----
# sales_24 = int(all_sales[(all_sales['date'] > '2023-12-31') & (all_sales['date'] < current_date)].usd.sum())
# sales_23 = int(all_sales[(all_sales['date'] > '2022-12-31') & (all_sales['date'].dt.date < year_ago_today.date())].usd.sum())
# delta = sales_24 - sales_23

df_selection = all_sales.copy()

In [ ]:
# print(sales_23)
# print(sales_24)
# print(current_date)
# print(year_ago_today)
# print(datetime.datetime.today())

In [ ]:
############

market_segment_color = {
    'Vending': 'rgb(56,149,73)',
    'Grocery': 'rgb(248,184,230)',
    'Alternate Retail': 'rgb(46,70,166)',
    'Canada': 'rgb(204,208,221)',
    'Online': 'rgb(106,87,63)',
    'Other': 'rgb(200,237,233)',
    'Convenience': 'rgb(233,81,46)',
    'Broadline Distributor': 'rgb(233,152,19)',
    'Samples': 'rgb(141,62,92)'}

In [ ]:
# segment pie chart

seg_sales = round(df_selection.groupby('market_segment',as_index=False)['usd'].sum()).sort_values(by='market_segment',ascending=False)

fig_seg_sales = px.pie(
    seg_sales, 
    values='usd', 
    names='market_segment',
    template = 'simple_white',
    title=' ',
    opacity = .8,
    hole=.33,
    hover_data = ['market_segment'],
    color='market_segment',
    color_discrete_map=market_segment_color).update_layout(autosize=False,width=450,height=450,showlegend=False)
fig_seg_sales.update_traces(textposition='inside', textinfo='percent+label', texttemplate='%{label}<br>%{percent:.0%}')#'{percent:.0%}')
fig_seg_sales.show()

In [ ]:
# df_selection = df_selection.reset_index(drop=True)
df_selection.head(3)

In [ ]:
# df_selection.index = pd.to_datetime(df_selection['date'],format = '%m/%d/%y')
# mth_sales = round(df_selection.groupby(pd.Grouper(freq='M'))['usd'].sum())

In [ ]:
df = pd.DataFrame(df_selection.groupby(['date','parent_customer'])['usd'].sum()).sort_values('date',ascending=False).reset_index().set_index('date')
df#.sort_values('date',ascending=False)#.reset_index(drop=True)

In [ ]:
df_selection.index = pd.to_datetime(df_selection['date'],format = '%y/%m/%d')

sales_per_week = df_selection.groupby(pd.Grouper(freq='W'))['usd'].sum()

In [ ]:
sales_per_week

In [ ]:
#all sales weekly scatter
fig_scatter_all = px.scatter(
    round(sales_per_week),
    x=sales_per_week.index,
    y='usd',
    title='<b>Weekly Sales</b>',
    template = 'presentation',
    labels={'date':'',
            'usd':'<b>$USD</b>'},
    height=425,
    size='usd',
    size_max=15,
    color='usd',
    color_continuous_scale=px.colors.sequential.Oranges,
    trendline="rolling", trendline_options=dict(function="mean", window=10), trendline_scope="overall", trendline_color_override="black"
)

fig_scatter_all.update_coloraxes(showscale=False)
fig_scatter_all.update_yaxes(gridcolor='lightgray')

fig_scatter_all.update_layout(
    title_x=0.4,
    hovermode='x unified',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        # title = '10 Week Moving Average'
    )
)

fig_scatter_all.update_traces(
    hovertemplate=
    "Sales: <b>%{y:$,.0f}")

fig_scatter_all.show()

In [ ]:
df_selection.info()

In [ ]:
# round(scatter_df
#  .usd
#  .loc['2021':]
#  .resample('W')
#  .mean()
# )

In [ ]:
# scatter_df = scatter_df.reset_index().groupby(['market_segment','date'],as_index=False)['usd'].sum().set_index('date')
# scatter_df.head(3)

In [ ]:
def make_daily_bar_df(df_selection):
    return (
        df_selection.dropna()
        .assign(new_date=pd.to_datetime(df_selection['date']))
        .drop(columns=['cad','month','year','date'])
        .rename(columns={'new_date':'date'})
        .groupby(['market_segment','date'],as_index=False)
        .sum()
        .set_index('date')
        .sort_index()
    )

daily_bar_df = make_daily_bar_df(df_selection)

In [ ]:
def make_daily_bar_df(df_selection):
    return (
        df_selection.dropna()
        .assign(new_date=pd.to_datetime(df_selection['date']))
        .drop(columns=['cad','month','year','date'])
        .rename(columns={'new_date':'date'})
        .groupby(['market_segment','date'],as_index=False)
        .sum()
        .set_index('date')
        .sort_index()
    )

daily_bar_df = make_daily_bar_df(df_selection)

week_ago = datetime.today().date() - pd.offsets.Day(8)
daily_bar_df = daily_bar_df[daily_bar_df.index > week_ago].sort_index()


px.scatter(daily_bar_df,
           x='usd',
           color='market_segment',
           color_discrete_map=market_segment_color,
           labels={'date':"", 'usd':'Daily Sales'},
           hover_name=daily_bar_df.loc['2024/1/23':].index.date,
           height=800,
           width=500
          ).update_traces(marker_size=25)

In [15]:
def make_daily_bar_df(df_selection):
    return (
        df_selection.dropna()
        .assign(new_date=pd.to_datetime(df_selection['date']))
        .drop(columns=['cad','month','year','date'])
        .rename(columns={'new_date':'date'})
        .groupby(['market_segment','date'],as_index=False)
        .sum()
        .set_index('date')
        .sort_index()
    )

daily_bar_df = make_daily_bar_df(df_selection)

week_ago = datetime.today().date() - pd.offsets.Day(15)
daily_bar_df = daily_bar_df[daily_bar_df.index > week_ago].sort_index()

C:\Users\mikej\AppData\Local\Temp\ipykernel_93268\2768862726.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()


In [39]:
daily = pd.DataFrame(daily_bar_df.groupby('date')['usd'].sum()).sort_index(ascending=False)
last10 = round(daily).usd.values#.sort_values('date',ascending=False)

In [40]:
last10

array([145711.,  67524.,  35374.,  32857.,  26417.,  30640.,  27740.,
        24112.,  31079.,  20633.])